<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun  1 18:36:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os 
path_drive = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet-Padel/'

Mounted at /content/gdrive


In [3]:
os.chdir('/content/')
!pwd

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

/content
Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 780, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 780 (delta 26), reused 94 (delta 20), pack-reused 680
Receiving objects: 100% (780/780), 862.27 MiB | 26.44 MiB/s, done.
Resolving deltas: 100% (325/325), done.
Checking out files: 100% (234/234), done.
Already up to date.


In [4]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 7.2 MB/s 
     |████████████████████████████████| 6.7 MB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 57.8 MB/s 
     |████████████████████████████████| 9.8 MB 57.3 MB/s 
     |████████████████████████████████| 34.5 MB 1.3 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=ce8f407e6a8e4d8b5491d6683f4299cc47f38f44d63c60f02ce82ba76f0b5245
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled sta

### Predict

In [ ]:
for i in range(10):
  video = '/content/gdrive/MyDrive/PadelMatchs/PremierRoma_Semis_LGBC_0_54_'+str(i)+'.mp4'
  !python predict.py --video_path={video} 

2022-05-29 18:45:26.314328: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Number of frames in video and dictionary are not the same!
Fail to load, predict only.
Beginning predicting......
Traceback (most recent call last):
  File "predict.py", line 116, in <module>
    y_pred = model.predict(img_input, batch_size=BATCH_SIZE)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 4125, in _get_operation_by_name_unsafe
    return self._nodes_by_name[name]
KeyError: 'res_net__track/sequential_10/res_net__bottle_neck_15/batch_normalization_56/FusedBatchNormV3'
2022-05-29 18:45:32.777034: W tensorflow/core/common_runtime/gpu/gpu_bfc_allo

### Test accuracy

In [22]:
file_name = 'Final_PremierQatar_Tie'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean3'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --load_weights={weights_path} # 2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-06-01 06:48:49.152945: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 144
Number of true negative: 64
Number of false positive FP1: 11
Number of false positive FP2: 3
Number of false negative: 166
Accuracy: 0.5360824742268041
Precision: 0.9113924050632911
Recall: 0.4645161290322581
Total Time: 32.25340986251831
(ACC + Pre + Rec)/3: 0.637330336107451
Done......
raw_data/points/Final_PremierQatar_Tie_10.mp4
2022-06-01 06:49:32.997802: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting..

### Train Padel

In [5]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/clean/*_0\.*
raw_data/points/clean/*_1\.*
raw_data/points/clean/*_2\.*
raw_data/points/clean/*_3\.*
raw_data/points/clean/*_4\.*
raw_data/points/clean/*_5\.*
raw_data/points/clean/*_6\.*
raw_data/points/clean/*_7\.*
raw_data/points/clean/*_8\.*


In [6]:
!rm -rf train_data/*
!python video2img.py 

==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_1.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_2.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_4.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_6.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_7.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_00_8.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_01_0.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_01_1.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_01_2.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_01_3.mp4
==========Convert End==========
Convert Video: raw_data/PremierRoma_Semis_LGBC_1_02_0.mp4
==========

*   TrackNet-PadelClean2: Entrenado solo con padel, pocos frames (5000) limpios (solo fragmentos con la bola en juego). --lr=10
*   TrackNet-PadelClean3: Continuación, 50 epochs --batch_size=5, con 7000 frames (alcanza 0.956 en la validación. - loss:1.20e-4)
*   TrackNet-PadelClean4: Continuación, +50epochs --bacth_size=5, con 12000 frames, --lr=1 (alcanza 0.94 en la validación. - loss:1.18e-4)

In [7]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean4'
!python train.py --save_weights={weights_path}  --batch_size=5 --lr=10  --pre_trained=True  --load_weights={weights_path}

lerning rate: 10.0
2022-06-01 18:40:42.729072: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 2256, Learning rate: 1.0000==========
2256/2256 [==============================] - 896s 391ms/step - loss: 1.4994e-04 - binary_accuracy: 0.9996
==========Epoch 0 start validation==========
Epoch 0 accuracy: 0.852
Epoch 0 precision: 0.981
Epoch 0 recall: 0.847
Epoch 0 average = (accuracy + precision + recall)/3: 0.893
==========Epoch 1, Train steps: 2256, Learning rate: 1.0000==========
2256/2256 [==============================] - 878s 389ms/step - loss: 1.4660e-04 - binary_accuracy: 0.9996
==========Epoch 1 start validation==========
Epoch 1 accuracy: 0.848
Epoch 1 precision: 0.977
Epoch 1 recall: 0.856
Epoch 1 average = (accuracy + precision + recall)/3: 0.894
==========Epoch 2, Train steps: 2256